In [ ]:
# default_exp readuniprot

# readuniprot

> Parse .dat files (Uniprot/Swissprot flat file format)

In [ ]:
#hide
%load_ext autoreload
%autoreload 2
from nbdev import show_doc

In [ ]:
#export

import collections
import csv
import itertools
import random
import re

from proteinscan import utils

In [ ]:
# export

def iterDat(datFPath) :
    """
    Iterator yielding a list of lines for each entry in the .dat file.
    """
    with utils.openGzipOrText(datFPath) as f :
        for (isEnd,it) in itertools.groupby(f, lambda ln : ln.strip()=='//') :
            if not isEnd :
                yield(list(it))

def datEntryLnsWithCode(datEntry,code) :
    "Returns a list of the lines in a .dat file entry with the given code."
    res = []
    for ln in datEntry :
        ln = ln.strip()
        if ln.startswith(code+' ') :
            res.append(ln[len(code):].lstrip())
    return res

def datEntryPrimaryAC(datEntry) :
    "Returns the primary accession number from a .det file entry."
    acLns = datEntryLnsWithCode(datEntry,'AC')
    return acLns[0].split(';')[0]

In [ ]:
entries = list(iterDat('uniprotTest.dat.gz'))
assert (len(entries)==185
        and len(entries[2])==61
        and entries[2][0]=='ID   002R_IIV3               Reviewed;         458 AA.\n'
        and entries[2][10]=='OX   NCBI_TaxID=345201;\n'
        and entries[2][-1]=='     QSIDRYFCSL DSNYNSEDED FEYDSDSEDD DSDSEDDC\n'
        and datEntryLnsWithCode(entries[2],'DT') ==
            ['16-JUN-2009, integrated into UniProtKB/Swiss-Prot.',
             '11-JUL-2006, sequence version 1.',
             '02-JUN-2021, entry version 28.']
        and datEntryPrimaryAC(entries[2])=='Q197F8')

In [ ]:
# export

def scanDat(datFPath, fn, returnFull=False, **kwargs) :
    """
    Scan a function across each entry in a .dat file, accumulating the results.
    If the function returns None, the entry is ignored; otherwise the result
    is added to the returned results list. Accepts additional keyword arguments,
    which are passed on to the function to be scanned.

    If returnFull is True, also returns a list of the corresponding full entries.
    """
    res = []
    if returnFull : fullRes = []
    for datEntry in iterDat(datFPath) :
        item = fn(datEntry, **kwargs)
        if item is not None :
            res.append(item)
            if returnFull :
                fullRes.append(datEntry)
    return (res,fullRes) if returnFull else res

def allPrimaryACsInDat(datFPath) :
    "Returns a list of all primary accession numbers in a .dat file."
    return scanDat(datFPath,datEntryPrimaryAC)

def datEntryPE(datEntry) :
    "Returns the level of evidence for a protein's existence (1-5, lower is stronger)."
    peLns = datEntryLnsWithCode(datEntry,'PE')
    return int(peLns[0][0])

def datEntryName(datEntry) :
    "Returns a descriptive name from a .dat file entry."
    deLns = datEntryLnsWithCode(datEntry,'DE')
    name = deLns[0].split('=')[1].rstrip(';')
    flagSet = set()
    for deLn in deLns[1:] :
        if deLn.startswith('Flags') :
            flagSet.update(flag.rstrip(';') for flag in deLn.split()[1:])
    if len(flagSet) > 0 :
        name += ' (' + '; '.join(sorted(flagSet)) +')'
    return name

def datEntrySeq(datEntry) :
    "Returns the amino acid sequence from a .dat file entry as a single string."
    seqL = []
    for ln in reversed(datEntry) :
        ln = ln.strip()
        if ln.startswith('SQ ') :
            break
        seqL.append(ln.replace(' ',''))
    return ''.join(reversed(seqL))

In [ ]:
acs = allPrimaryACsInDat('uniprotTest.dat.gz')
pes = scanDat('uniprotTest.dat.gz',datEntryPE)
names = scanDat('uniprotTest.dat.gz',datEntryName)
seqs = scanDat('uniprotTest.dat.gz',datEntrySeq)
assert (len(acs)==185
        and acs[:5]==['Q6GZX4', 'Q6GZX3', 'Q197F8', 'Q197F7', 'Q6GZX2']
        and acs[-5:]==['P0C9G3', 'P0C9G4', 'P0C9G1', 'P18559', 'P0C9G6']
        and len(pes)==185
        and pes[:5]==[4, 4, 4, 4, 3]
        and pes[-5:]==[3, 3, 3, 3, 3]
        and len(names)==185
        and names[:3]==['Putative transcription factor 001R',
                      'Uncharacterized protein 002L', 'Uncharacterized protein 002R']
        and names[-3:]==['Protein MGF 110-1L', 'Protein MGF 110-2L (Precursor)',
                         'Protein MGF 110-2L (Precursor)']
        and len(seqs)==185
        and len(seqs[0])==256
        and seqs[0]=='MAFSAEDVLKEYDRRRRMEALLLSLYYPNDRKLLDYKEWSPPRVQVECPKAPVEWNNPPSEKGLIVGH'
            +'FSGIKYKGEKAQASEVDVNKMCCWVSKFKDAMRRYQGIQTCKIPGKVLSDLDAKIKAYNLTVEGVEGFVRYSRVTK'
            +'QHVAAFLKELRHSKQYENVNLIHYILTDKRVDIQHLEKDLVKDFKALVESAHRMRQGHMINVKYILYQLLKKHGHG'
            +'PDGPDILTVKTGSKGVLYDDSFRKIYTDLGWKFTPL'
        and len(seqs[-1])==113
        and seqs[-1]=='MGFFSYLGLVLVGLASLTSLASLANLQDFSTDNPLEEELRCWCQYVKNCRFCWACQDGFCKNKVLK'
            +'NMPSVQEHSYPMEHCMIHRQCKYVRDGPIFQVECMMQTCDAIHLLNA')

In [ ]:
# export

def datEntryGOLines(datEntry) :
    "Returns the lines with GO terms from the database x-ref section of a .dat file entry."
    drLns = datEntryLnsWithCode(datEntry,'DR')
    return [drLn for drLn in drLns if drLn.startswith('GO;')]

def datEntryGOTermPresent(datEntry,goTerm) :
    "Returns 'pos' or 'neg' accordingly as a .dat file entry includes the given GO term."
    if any(goTerm in goLn for goLn in datEntryGOLines(datEntry)) :
        return 'pos'
    return 'neg'

def datEntryIsAtpBinding(datEntry) :
    "Returns 'pos' or 'neg accordingly as a .dat file entry has the ATP binding GO term."
    return datEntryGOTermPresent(datEntry,'GO:0005524;')
def datEntryIsGtpBinding(datEntry) :
    "Returns 'pos' or 'neg accordingly as a .dat file entry has the GTP binding GO term."
    return datEntryGOTermPresent(datEntry,'GO:0005525;')
def datEntryIsMetalBinding(datEntry) :
    "Returns 'pos' or 'neg accordingly as a .dat file entry has the metal binding GO term."
    return datEntryGOTermPresent(datEntry,'GO:0046872;')

def datEntryKWs(datEntry) :
    "Returns a list of the keywords from the KW section of a .dat file entry."
    res = []
    for kwLn in datEntryLnsWithCode(datEntry,'KW') :
        for kw in kwLn.split(';') :
            kw = kw.strip().rstrip('.').strip().lower()
            if kw != '' :
                res.append(kw)
    return res

In [ ]:
goLns = scanDat('uniprotTest.dat.gz',datEntryGOLines)
kws = scanDat('uniprotTest.dat.gz',datEntryKWs)
assert (goLns[0]==['GO; GO:0046782; P:regulation of viral transcription; IEA:InterPro.']
        and goLns[1]==['GO; GO:0033644; C:host cell membrane; IEA:UniProtKB-SubCell.',
                       'GO; GO:0016021; C:integral component of membrane; IEA:UniProtKB-KW.']
        and goLns[-1]==[]
        and datEntryGOTermPresent(entries[0],'GO:0046782')=='pos'
        and datEntryGOTermPresent(entries[1],'GO:0033644')=='pos'
        and datEntryGOTermPresent(entries[1],'GO:0016021')=='pos'
        and datEntryGOTermPresent(entries[1],'GO:0046782')=='neg'
        and datEntryGOTermPresent(entries[-1],'GO:0046782')=='neg'
        and kws[0]==['activator','reference proteome','transcription',
                     'transcription regulation']
        and kws[1]==['host membrane','membrane','reference proteome',
                     'transmembrane','transmembrane helix']
        and kws[-1]==['early protein', 'signal'])

In [ ]:
# export

aaLetters = 'ACDEFGHIKLMNPQRSTVWY'
aaLettersSet = set(aaLetters)

def listIfStr(lOrStr) :
    return [lOrStr] if isinstance(lOrStr,str) else lOrStr

def filterDatEntry(datEntry, restrictTo20AA=True,
            minLen=50, maxLen=400, maxPE=None,
            requireInSpecies='', elimKWs=[], requireKWs=[],
            requireInName='', excludeStrs=[]) :
    """
    Filters a .dat file entry for inclusion in a LM or classification data set.
    The filtering is based on the keyword arguments.

    Returns (primary_accession_num,sequence) if the entry passes the filter, else None.
    """
    seq = datEntrySeq(datEntry)
    kws = datEntryKWs(datEntry)
    elimKWs = listIfStr(elimKWs)
    requireKWs = listIfStr(requireKWs)
    excludeStrs = listIfStr(excludeStrs)
    if ((restrictTo20AA and any(c not in aaLettersSet for c in seq))
        or any(elimKW in kws for elimKW in elimKWs)
        or any(requireKW not in kws for requireKW in requireKWs)
        or (minLen is not None and len(seq)<minLen)
        or (maxLen is not None and len(seq)>maxLen)
        or (maxPE is not None and datEntryPE(datEntry)>maxPE)
        or requireInSpecies not in datEntryLnsWithCode(datEntry,'OS')[0].lower()) :
        return None
    lName = datEntryName(datEntry).lower()
    if ((requireInName not in lName)
        or any (excludeStr in lName for excludeStr in excludeStrs)) :
        return None
    return (datEntryPrimaryAC(datEntry),seq)

In [ ]:
assert (len(scanDat('uniprotTest.dat.gz',filterDatEntry))==166
        and len(scanDat('uniprotTest.dat.gz',filterDatEntry,maxPE=3))==50
        and len(scanDat('uniprotTest.dat.gz',filterDatEntry,maxPE=3,
                        requireKWs=['transmembrane']))==15)